<a href="https://colab.research.google.com/github/LoveTechCode/Modulo1/blob/master/Cesta_de_la_compra_Enrique_Cabello.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# La cesta de la compra .
Una vez estudiado el punto 3.6 del temario del curso de Machine Learnig, ha llegado el momento de estudiar este ejemplo.
 
El primer paso es obtener los datos con los que vamos a trabajar.
Los datos proceden de la Universidad de California, Irvine, que proporciona un conjunto de datos que contiene las transacciones de un sitio web basado en el Reino Unido.

Para esta tarea ejecuta la celda que tienes justo debajo. 

Pd. No está de más recordar que no trabajes sobre este cuaderno haz una copia del mismo y guardalá en tu Google Drive o disco duro. 
PD. Es muy recomendable que estudies previamente algún tutorial sencillo sobre la biblioteca Pandas. 


In [1]:
! curl http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx -o retail.xlsx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22.6M  100 22.6M    0     0  18.2M      0  0:00:01  0:00:01 --:--:-- 18.2M


Importamos las libreria Pandas y cargamos el archivo 'retail.xlsx'

In [2]:
import pandas as pd
df = pd.read_excel('retail.xlsx')
print( df.columns)
# puede que la ejecución del codigo tarde un poco


Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')


In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


Hemos comentado que el análisis de la cesta de la compra no requiere una destrucción de datos exhaustiva, pero a quí vamos a eliminar los números de factura nulos y los pedidos cancelados. También es útil quitar los espacios no pertinentes de las descripciones de producto y convertir todos los números de factura en cadenas. Para hacerlo, ejecuta el código siguiente. 

Investiga sobre aquellas funciones que no conozcas. 

In [6]:
# borra espacios en blanco al principio y al final del texto
df['Description'] = df['Description'].str.strip()
df['Description']

0          WHITE HANGING HEART T-LIGHT HOLDER
1                         WHITE METAL LANTERN
2              CREAM CUPID HEARTS COAT HANGER
3         KNITTED UNION FLAG HOT WATER BOTTLE
4              RED WOOLLY HOTTIE WHITE HEART.
                         ...                 
541904            PACK OF 20 SPACEBOY NAPKINS
541905            CHILDREN'S APRON DOLLY GIRL
541906           CHILDRENS CUTLERY DOLLY GIRL
541907        CHILDRENS CUTLERY CIRCUS PARADE
541908           BAKING SET 9 PIECE RETROSPOT
Name: Description, Length: 541909, dtype: object

In [8]:
# vamos a probar con un ejemplo sencillo
text = '  remove spaces   '
text = text.strip()
text

'remove spaces'

In [9]:
# elimina las filas donde en la columna 'InvoiceNo" hay un N/A como valor
df.dropna(axis = 0, subset=['InvoiceNo'], inplace = True)

In [13]:
# convierte los valores de la columna 'InvoiceNo' al tipo 'str'
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

In [16]:
# Devuelve solo aquellas entradas donde no haya una 'C' en la columna 'InvoiceNo'
df = df[~df['InvoiceNo'].str.contains('C')]

In [17]:
df['InvoiceNo']

0         536365
1         536365
2         536365
3         536365
4         536365
           ...  
541904    581587
541905    581587
541906    581587
541907    581587
541908    581587
Name: InvoiceNo, Length: 532621, dtype: object

En esta linea sumamos los datos y lo vemos a nivel de pais. 

In [20]:
df.groupby('Country').count().reset_index().sort_values('InvoiceNo', ascending = False).head()

,Country,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
36,United Kingdom,487622,487622,486167,487622,487622,487622,354345
14,Germany,9042,9042,9042,9042,9042,9042,9042
13,France,8408,8408,8408,8408,8408,8408,8342
10,EIRE,7894,7894,7894,7894,7894,7894,7238
31,Spain,2485,2485,2485,2485,2485,2485,2485


A continuación, reorganizaremos los datos con cada producto **codificado con codificación one-hot, y una transacción por fila**. 
**La codificación one-hot** es una técnica de transformación de datos donde los valores categoricos se convierten en valores 0-1. También centraremos el ámbito del conjunto de datos a un país, en este caso, Francia, para comparar el comportamiento del consumidor en un mercado individual. 
Escribe y ejecuta el código siguiente para hacerlo .

Pd. Investiga más acerca de la codificación one-hot como ejercicio. 

In [22]:
# Nos quedamos solo con los datos de la cesta de Francia
# Los agrupamos por InvoiceNo y Description
# Rellenamos con 0 las celdas donde no hay valor (NA)
basket_fr = (
          df[df['Country']=="France"].groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo')
          )
basket_fr.head(10)

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,15CM CHRISTMAS GLASS BALL 20 LIGHTS,16 PIECE CUTLERY SET PANTRY DESIGN,18PC WOODEN CUTLERY SET DISPOSABLE,20 DOLLY PEGS RETROSPOT,200 RED + WHITE BENDY STRAWS,3 HOOK HANGER MAGIC GARDEN,3 PIECE SPACEBOY COOKIE CUTTER SET,3 RAFFIA RIBBONS 50'S CHRISTMAS,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN RED AND CREAM,3 TRADITIONAl BISCUIT CUTTERS SET,36 DOILIES DOLLY GIRL,36 DOILIES VINTAGE CHRISTMAS,36 FOIL HEART CAKE CASES,36 FOIL STAR CAKE CASES,36 PENCILS TUBE RED RETROSPOT,36 PENCILS TUBE SKULLS,36 PENCILS TUBE WOODLAND,3D DOG PICTURE PLAYING CARDS,3D HEARTS HONEYCOMB PAPER GARLAND,3D SHEET OF DOG STICKERS,3D TRADITIONAL CHRISTMAS STICKERS,3D VINTAGE CHRISTMAS STICKERS,4 IVORY DINNER CANDLES SILVER FLOCK,4 PINK DINNER CANDLE SILVER FLOCK,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER MAGIC TOADSTOOL,5 HOOK HANGER RED MAGIC TOADSTOOL,50'S CHRISTMAS GIFT BAG LARGE,6 GIFT TAGS 50'S CHRISTMAS,...,WOODLAND DESIGN COTTON TOTE BAG,WOODLAND LARGE BLUE FELT HEART,WOODLAND LARGE PINK FELT HEART,WOODLAND LARGE RED FELT HEART,WOODLAND MINI BACKPACK,WOODLAND PARTY BAG + STICKER SET,WOODLAND SMALL BLUE FELT HEART,WOODLAND SMALL PINK FELT HEART,WOODLAND SMALL RED FELT HEART,WOODLAND STORAGE BOX LARGE,WOODLAND STORAGE BOX SMALL,WORLD WAR 2 GLIDERS ASSTD DESIGNS,WRAP VINTAGE DOILY,WRAP 50'S CHRISTMAS,WRAP ALPHABET DESIGN,WRAP CAROUSEL,WRAP CHRISTMAS VILLAGE,WRAP CIRCUS PARADE,WRAP DOILEY DESIGN,WRAP DOLLY GIRL,WRAP ENGLISH ROSE,WRAP GINGHAM ROSE,WRAP GREEN PEARS,WRAP I LOVE LONDON,WRAP PAISLEY PARK,WRAP PINK FAIRY CAKES,WRAP POPPIES DESIGN,WRAP RED APPLES,WRAP RED VINTAGE DOILY,WRAP SUKI AND FRIENDS,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537468,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

Observando los resultados vemos un problema con la codificación one-hot. El sexto elemento tiene un valor de 24.0 en la segunda columna. La intención es tener un uno o un cero en los datos, no cantidades. Por lo tanto, tendremos que localizar cualquier valor distinto de cero y convertirlo en 1. Para corregir este problema, ejecuta el siguiente código y ten en cuenta que se ha convertido el valor 24,0 en un 1:

In [23]:
def sum_to_boolean(x):
  if x<=0:
    return 0
  else:
    return 1
basket_fr_final = basket_fr.applymap(sum_to_boolean)
basket_fr_final.head(10)

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,15CM CHRISTMAS GLASS BALL 20 LIGHTS,16 PIECE CUTLERY SET PANTRY DESIGN,18PC WOODEN CUTLERY SET DISPOSABLE,20 DOLLY PEGS RETROSPOT,200 RED + WHITE BENDY STRAWS,3 HOOK HANGER MAGIC GARDEN,3 PIECE SPACEBOY COOKIE CUTTER SET,3 RAFFIA RIBBONS 50'S CHRISTMAS,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN RED AND CREAM,3 TRADITIONAl BISCUIT CUTTERS SET,36 DOILIES DOLLY GIRL,36 DOILIES VINTAGE CHRISTMAS,36 FOIL HEART CAKE CASES,36 FOIL STAR CAKE CASES,36 PENCILS TUBE RED RETROSPOT,36 PENCILS TUBE SKULLS,36 PENCILS TUBE WOODLAND,3D DOG PICTURE PLAYING CARDS,3D HEARTS HONEYCOMB PAPER GARLAND,3D SHEET OF DOG STICKERS,3D TRADITIONAL CHRISTMAS STICKERS,3D VINTAGE CHRISTMAS STICKERS,4 IVORY DINNER CANDLES SILVER FLOCK,4 PINK DINNER CANDLE SILVER FLOCK,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER MAGIC TOADSTOOL,5 HOOK HANGER RED MAGIC TOADSTOOL,50'S CHRISTMAS GIFT BAG LARGE,6 GIFT TAGS 50'S CHRISTMAS,...,WOODLAND DESIGN COTTON TOTE BAG,WOODLAND LARGE BLUE FELT HEART,WOODLAND LARGE PINK FELT HEART,WOODLAND LARGE RED FELT HEART,WOODLAND MINI BACKPACK,WOODLAND PARTY BAG + STICKER SET,WOODLAND SMALL BLUE FELT HEART,WOODLAND SMALL PINK FELT HEART,WOODLAND SMALL RED FELT HEART,WOODLAND STORAGE BOX LARGE,WOODLAND STORAGE BOX SMALL,WORLD WAR 2 GLIDERS ASSTD DESIGNS,WRAP VINTAGE DOILY,WRAP 50'S CHRISTMAS,WRAP ALPHABET DESIGN,WRAP CAROUSEL,WRAP CHRISTMAS VILLAGE,WRAP CIRCUS PARADE,WRAP DOILEY DESIGN,WRAP DOLLY GIRL,WRAP ENGLISH ROSE,WRAP GINGHAM ROSE,WRAP GREEN PEARS,WRAP I LOVE LONDON,WRAP PAISLEY PARK,WRAP PINK FAIRY CAKES,WRAP POPPIES DESIGN,WRAP RED APPLES,WRAP RED VINTAGE DOILY,WRAP SUKI AND FRIENDS,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
537468,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
537693,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
537897,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
537967,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Para analizar los datos aún más, utilizaremos MLXTEND 
[http://rasbt.github.io/mlxtend/](http://rasbt.github.io/mlxtend/)

MLXTEND es una extensión de Python de herramientas útiles para tareas de ciencia de datos como el análisis de la cesta de la compra. 
Para instalar esta biblioteca desde el cuaderno, ejecuta el código siguiente:
Pd. Como ejercicio sería muy interesante que revisaras esta biblioteca con más detenimiento. 

In [24]:
pip install mlxtend

Una vez instalado el paquete MLXTEND, es hora de importar las bibliotecas relevantes de MLXTEND, como se indica a continuación:
Pd. Es muy recomendable que investigues sobre estas importaciones. 

In [27]:
#--- apriori ---
# apriori(df, min_support=0.5, use_colnames=False, max_len=None, verbose=0, low_memory=False)
# Returns:
# pandas DataFrame with columns ['support', 'itemsets'] of all itemsets that are >= min_support and < than max_len (if max_len is not None)
# Each itemset in the 'itemsets' column is of type frozenset, 
# which is a Python built-in type that behaves similarly to sets except that it is immutable

#--- association_rules ---
# association_rules(df, metric='confidence', min_threshold=0.8, support_only=False)
# Metric to evaluate if a rule is of interest. 
# Automatically set to 'support' if support_only=True
# Otherwise, supported metrics are 'support', 'confidence', 'lift','leverage', and 'conviction' 


from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori


En esta celda, escribe el código siguiente para ver conjuntos de artículos con un soporte de, al menos, el 6 por ciento:

In [28]:
# nos quedamos con los que son mayores de > 0.06
frequent_itemsets_fr = apriori(basket_fr_final, min_support = 0.06, use_colnames = True)
frequent_itemsets_fr

,support,itemsets
0,0.071429,(4 TRADITIONAL SPINNING TOPS)
1,0.096939,(ALARM CLOCK BAKELIKE GREEN)
2,0.102041,(ALARM CLOCK BAKELIKE PINK)
3,0.094388,(ALARM CLOCK BAKELIKE RED)
4,0.068878,(ASSORTED COLOUR MINI CASES)
...,...,...
124,0.084184,"(POSTAGE, SET/6 RED SPOTTY PAPER CUPS, SET/20 ..."
125,0.084184,"(SET/6 RED SPOTTY PAPER PLATES, POSTAGE, SET/2..."
126,0.102041,"(SET/6 RED SPOTTY PAPER PLATES, POSTAGE, SET/6..."
127,0.099490,"(SET/6 RED SPOTTY PAPER PLATES, SET/6 RED SPOT..."


In [29]:
# los ordenamos de mayor a menor en 'support'
frequent_itemsets_fr.sort_values('support', ascending = False).head()

,support,itemsets
34,0.765306,(POSTAGE)
35,0.188776,(RABBIT NIGHT LIGHT)
41,0.181122,(RED TOADSTOOL LED NIGHT LIGHT)
32,0.170918,(PLASTERS IN TIN WOODLAND ANIMALS)
29,0.168367,(PLASTERS IN TIN CIRCUS PARADE)


Con los conjuntos de artículos clave identificados, ya podemos aplicar la biblioteca de reglas de asociación para detectar los comportamientos de compra de los clientes. Escribe el código siguiente y ejecutalo:

In [30]:
# metrics:

# support(A->C) = support(A+C) [aka 'support'], range: [0, 1]
# confidence(A->C) = support(A+C) / support(A), range: [0, 1]
# lift(A->C) = confidence(A->C) / support(C), range: [0, inf]
# leverage(A->C) = support(A->C) - support(A)*support(C), range: [-1, 1]
# conviction = [1 - support(C)] / [1 - confidence(A->C)], range: [0, inf]

a_rules = association_rules(frequent_itemsets_fr, metric = "lift", min_threshold = 1)
a_rules.sort_values('lift',ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
14,(CHILDRENS CUTLERY SPACEBOY),(CHILDRENS CUTLERY DOLLY GIRL),0.068878,0.071429,0.063776,0.925926,12.962963,0.058856,12.535714
15,(CHILDRENS CUTLERY DOLLY GIRL),(CHILDRENS CUTLERY SPACEBOY),0.071429,0.068878,0.063776,0.892857,12.962963,0.058856,8.690476
111,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",(ALARM CLOCK BAKELIKE RED),0.073980,0.094388,0.063776,0.862069,9.133271,0.056793,6.565689
114,(ALARM CLOCK BAKELIKE RED),"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",0.094388,0.073980,0.063776,0.675676,9.133271,0.056793,2.855230
122,"(POSTAGE, ALARM CLOCK BAKELIKE GREEN)",(ALARM CLOCK BAKELIKE RED),0.084184,0.094388,0.071429,0.848485,8.989353,0.063483,5.977041
...,...,...,...,...,...,...,...,...,...
44,(POSTAGE),(MINI PAINT SET VINTAGE),0.765306,0.104592,0.081633,0.106667,1.019837,0.001588,1.002323
94,(POSTAGE),(SPACEBOY LUNCH BOX),0.765306,0.125000,0.096939,0.126667,1.013333,0.001276,1.001908
95,(SPACEBOY LUNCH BOX),(POSTAGE),0.125000,0.765306,0.096939,0.775510,1.013333,0.001276,1.045455
36,(POSTAGE),(LUNCH BAG SPACEBOY DESIGN),0.765306,0.119898,0.091837,0.120000,1.000851,0.000078,1.000116


Los resultados mostrados arriba que están ordenados por mejora, revelan los patrones de compra de los clientes franceses en el sitio. Un vistazo rápido a los resultados muestra que los clientes que compran CHILDRENS CUTLERY SPACEBOY también adquieren CHILDRENS CUTLERY DOLLY GIRL, y que los clientes que compran un reloj de alarma en un color también lo compran en otro color. 
Con estos datos podríamos recomendar a los propietarios de los sitios ofrecer descuentos en conjuntos de productos para cuberterias por género, así como conjuntos de relojes de alarma en varios colores.

Sin embargo, tenga en que esta lista está ordenada por mejora, y no por repetición. Entonces puede que no tenga sentido introducir un nuevo conjunto u oferta de productos sin saber si ese producto es lo suficientemente popular. Para obtener una visión de la popularidad de estos productos de cubertería, escribimos el código Python siguiente:

In [31]:
print( basket_fr_final['CHILDRENS CUTLERY SPACEBOY'].sum())
print( basket_fr_final['CHILDRENS CUTLERY DOLLY GIRL'].sum())

27
28


Los resultados no son prometedores: solo 27 para SPACEBOY y 28 para DOLLY GIRL. Seguramente habréis observado que hay dos métricas más en la trama de datos resultante: Apalancamiento y convicción. 
Estos son los valores adicionales que se deben tener en cuenta al realizar el análisis de la cesta de la compra. 
Para obtener más información sobre esto, investiga las llamadas:

** "medidas alternativas de grado de interés"**

# metrics:

- support(A->C) = support(A+C) [aka 'support'], range: [0, 1]
- confidence(A->C) = support(A+C) / support(A), range: [0, 1]
- lift(A->C) = confidence(A->C) / support(C), range: [0, inf]
- leverage(A->C) = support(A->C) - support(A)*support(C), range: [-1, 1]
- conviction = [1 - support(C)] / [1 - confidence(A->C)], range: [0, inf]

# Resumen
En este cuaderno, hemos explorado el uso del análisis de la cesta de la compra para detectar patrones en el comportamiento de los consumidores. 
El análisis de la cesta de la compra pertenece al campo del análisis de afinidad, que usan las grandes empresas para conseguir que los consumidores se gasten más dinero en productos. 
La cesta de la compra suele ser un buen punto de partida para los principiantes en Machine Learnig, sus fundamentos matemáticos no son dificiles y puede ofrecer buenas oportunidades laborales. 